<a href="https://colab.research.google.com/github/LeeSeungwon89/Lecture-and-self-study/blob/master/5-2%20%EA%B5%90%EC%B0%A8%20%EA%B2%80%EC%A6%9D%EA%B3%BC%20%EA%B7%B8%EB%A6%AC%EB%93%9C%20%EC%84%9C%EC%B9%98(%EC%9E%91%EC%97%85%20%EC%A4%91).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차 검증과 그리드 서치

## 데이터 준비

In [68]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')
print(wine)

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
print(data, target)

      alcohol  sugar    pH  class
0         9.4    1.9  3.51    0.0
1         9.8    2.6  3.20    0.0
2         9.8    2.3  3.26    0.0
3         9.8    1.9  3.16    0.0
4         9.4    1.9  3.51    0.0
...       ...    ...   ...    ...
6492     11.2    1.6  3.27    1.0
6493      9.6    8.0  3.15    1.0
6494      9.4    1.2  2.99    1.0
6495     12.8    1.1  3.34    1.0
6496     11.8    0.8  3.26    1.0

[6497 rows x 4 columns]
[[ 9.4   1.9   3.51]
 [ 9.8   2.6   3.2 ]
 [ 9.8   2.3   3.26]
 ...
 [ 9.4   1.2   2.99]
 [12.8   1.1   3.34]
 [11.8   0.8   3.26]] [0. 0. 0. ... 1. 1. 1.]


In [69]:
# 훈련 세트와 테스트 세트로 나눔.
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)
print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


## 검증 세트

- 현재까지는 훈련 세트로 모델을 훈련하고 테스트 세트로 모델을 평가했음. 그러나 테스트 세트만으로 계속 성능을 확인하다 보면 점점 테스트 세트에 일반화 성능을 맞추게 되는 셈임.

- 테스트 세트로 일반화 성능을 올바르게 예측하려면 모델을 만들고 나서 마지막에 딱 한 번만 사용하는 것이 좋음.

- 그러나 테스트 세트를 사용하지 않으면 모델이 과대적합인지 과소적합인지 판단하기 어려움. 테스트 세트를 사용하지 않고 측정하는 방법은 훈련 세트를 또 나누는 것임. 이렇게 나눈 데이터를 '검증 세트(validation set)' 혹은 '개발 세트(dev set)' 라고 부름. 실제로 많이 사용하는 방법임.

 1. 훈련 세트 80%, 테스트 세트 20%로 나누고 훈련 세트에서 다시 20%를 떼서 검증 세트로 만드는 식임. 보통 테스트 세트와 검증 세트를 20 ~ 30%만큼 떼지만, 훈련 세트가 아주 많다면 몇 %만 떼어 놓아도 전체 데이터를 대표하는 데 문제가 없음.

 1. 훈련 세트로 모델을 훈련한 후 검증 세트로 모델을 평가함. 이런 방식으로 테스트 하고자 하는 매개변수를 바꿔가며 가장 좋은 모델을 고름. 그리고 나서 이 매개변수를 사용하여 훈련 세트와 검증 세트를 합친 전체 훈련 데이터로 모델을 다시 훈련함.

 1. 마지막에 테스트 세트로 최종 점수를 평가함. 실전에서 테스트 세트 점수와 비슷한 성능을 기대할 수 있음.

### 검증 세트 만들기

In [70]:
# train_input과 train_target을 다시 train_test_split() 함수에 넣어서,
# 훈련 세트를 sub_input, sub_target으로 만들고,
# 검증 세트를 val_input, val_target으로 만듦.
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)
print(sub_input.shape, val_input.shape)
# 훈련 세트의 샘플 5197개를 나눠서 훈련 세트 80%와 검증 세트 20%로 만들었음.

(4157, 3) (1040, 3)


### 결정 트리 모델 만들기

In [71]:
# 훈련 세트와 검증 세트로 결정 트리 모델을 훈련함.
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state = 42)

dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
# 과대적합 된 상태이므로 매개변수를 바꿔서 더 좋은 모델을 찾아야 함.

0.9971133028626413
0.864423076923077


## 교차 검증

- 검증 세트를 만드느라 훈련 세트의 총량이 줄었음. 보통 많은 데이터를 훈련에 사용할수록 좋은 모델이 만들어지지만, 그렇다고 검증 세트를 너무 조금 떼면 검증 점수가 불안정할 것임. 이런 경우에 '교차 검증(cross validation)' 을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있음.

 1. 검증 세트를 떼서 평가하는 과정을 여러번 반복함.
 
 1. 마지막으로 이 점수들을 평균하여 최종 검증 점수를 얻음.

- k-폴드(겹) 교차 검증(k-fold cross validation): 훈련 세트를 k만큼 부분으로 나눠서 교차 검증을 수행하는 방법. 보통 5나 10으로 나누는 5-폴드 교차 검증, 10-폴드 교차 검증을 많이 사용함. 이렇게 하면 데이터의 80 ~ 90%까지 훈련에 사용할 수 있음. 검증 세트는 줄지만 각 폴드에서 계산한 검증 점수를 평균하기 때문에 안정된 점수로 판단할 수 있음.

- 3-폴드 교차 검증을 예로 들면, 아래처럼 훈련 세트를 세 부분으로 나누고, 훈련 세트로는 모델을 훈련하고, 검증 세트로는 모델을 평가함.

 1. 훈련 세트, 훈련 세트, 검증 세트
 
 1. 훈련 세트, 검증 세트, 훈련 세트
 
 1. 검증 세트, 훈련 세트, 훈련 세트

- cross_validate(): 교차 검증을 수행하는 함수.

 - scoring: 검증에 사용할 평가 지표를 지정하는 매개변수. 기본값은 분류 모델은 정확도인 'accuracy', 회귀 모델은 결정계수인 'r2'.

 - cv: 교차 검증 폴드 수나 스플리터 객체를 지정하는 매개변수. 기본값은 5. 자세한 설명은 아래에서 제시함.

 - n_jobs: 교차 검증을 수행할 때 사용할 CPU 코어 수를 지정하는 매개변수. 기본값은 1이며 -1로 지정하면 모든 시스템 코어를 사용함.

 - return_train_score: True로 지정하면 훈련 세트의 점수도 반환하는 매개변수. 기본값은 False.

In [72]:
# 'cross_validate()' 함수를 사용하여 교차 검증을 실시함. 
# 먼저 평가할 모델 객체(결정 트리를 의미하는 dt)를 첫 번째 매개변수로 전달하고,
# 직접 검증 세트를 떼지 않고 훈련 세트 전체를 두 세 번째 매개변수에 전달함.
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)
# 'fit_time', 'score_time', 'test_score' 키를 가진 딕셔너리를 반환함.
# fit_time: 훈련하는 시간.
# score_time: 검증하는 시간.
# test_score: 검증 폴드의 점수. 출력된 점수 5개를 평균하여 교차 검증의 최종 점수를 얻을 수 있음.

{'fit_time': array([0.00705719, 0.00693512, 0.00777507, 0.00709319, 0.00674462]), 'score_time': array([0.00063396, 0.00056696, 0.00058794, 0.00059319, 0.00052738]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


### 최상의 검증 점수 확인하기

In [73]:
# 'test_score' 의 점수 5개를 평균하여 교차 검증의 최상의 점수를 확인함.
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


### 분할기로 폴드 나누기

In [74]:
# 위에서 'train_test_split()' 함수로 전체 데이터를 섞었지만
# 본래 'cross_validate()' 함수는 훈련 세트를 섞고 폴드를 나누지 않음.
# 따라서 교차 검증을 할 때 훈련 세트를 섞으려면 'cv' 매개변수에 '분할기(splitter)' 를 지정해야 함.
# 분할기는 교차 검증에서 폴드를 어떻게 나눌지 결정하며,
# 회귀 모델일 경우 'KFold분할기' 를 사용하고,
# 분류 모델일 경우 'StratifiedKFold' 를 사용함.
# 즉 앞서 수행한 교차 검증은 아래처럼 표현함.
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [75]:
# 훈련 세트를 섞은 후 '10-폴드 교차 검증' 을 수행함.
splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
# n_splits: 훈련 세트를 몇 부분으로 나눌지 결정하는 매개변수. 몇(k) 폴드 교차 검증을 할지 결정함. 기본값은 5.
# shuffle: 세트의 샘플을 섞을지 결정하는 매개변수. 기본값은 False.

scores = cross_validate(dt, train_input, train_target, cv = splitter)
print(np.mean(scores['test_score']))
# 'KFold' 클래스도 동일한 방식으로 사용함.
# 5-폴드와 10-폴드의 차이가 크지 않음.

0.8574181117533719


## 하이퍼파라미터 튜닝

- 하이퍼파라미터(Hyper parameter): 모델이 학습할 수 없어서 사용자가 직접 지정해야 하는 파라미터. 클래스나 메서드의 매개변수로 표현됨. 먼저 라이브러리가 제공하는 기본값으로 훈련하고, 매개변수를 조금씩 바꾸면서 모델을 훈련하고 교차 검증을 수행함. 'AutoML' 는 사람의 개입 없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술임.

 - 가령 결정 트리 모델에서 'max_depth' 매개변수의 최적값을 찾았다고 가정했을 때, 'max_depth' 값을 고정한 상태로 'min_samples_split' 매개변수의 최적값을 찾아가는 과정은 옳지 않은 방법임. 'max_depth' 매개변수의 최적값은 'min_samples_split' 매개변수의 값이 바뀌면 함께 달라지므로 두 매개변수의 값을 동시에 바꾸면서 최적값을 찾아야 함.

 - 매개변수가 많아지면 최적값을 일일이 찾기 더 복잡해짐. 이 과정을 아래에서 설명하는 '그리드 서치(Grid Search)' 클래스가 도와줌.

### 그리드 서치

- GridSearchCV: 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행하는 클래스. 별도로 'cross_validate()' 함수를 호출할 필요가 없음. 교차 검증 함수(cross_validate())의 매개변수와 동일함.

 - scoring: 검증에 사용할 평가 지표를 지정하는 매개변수. 기본값은 분류 모델은 정확도인 'accuracy', 회귀 모델은 결정계수인 'r2'.

 - cv: 교차 검증 폴드 수나 스플리터 객체를 지정하는 매개변수. 기본값은 5. 자세한 설명은 아래에서 제시함.

 - n_jobs: 교차 검증을 수행할 때 사용할 CPU 코어 수를 지정하는 매개변수. 기본값은 1이며 -1로 지정하면 모든 시스템 코어를 사용함.

 - return_train_score: True로 지정하면 훈련 세트의 점수도 반환하는 매개변수. 기본값은 False.

#### 그리드 서치로 매개변수 최적값 찾기

In [76]:
# 'GridSearchCV' 클래스로 'min_impurity_decrease' 매개변수의 최적값을 찾음.
# 참고로 결정 트리 클래스의 'min_impurity_decrease' 매개변수는 가지치기에 사용하는 매개변수로서
# 지정한 값보다 작으면 더이상 분할하지 않음.
from sklearn.model_selection import GridSearchCV

# 하이퍼파라미터를 'params' 라는 변수로 만듦.
# 'min_impurity_decrease' 매개변수를 0.0001 ~ 0.0005까지 리스트로 값을 부여하여 딕셔너리로 묶음.
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

# 'GridSearchCV' 클래스에 탐색 대상 모델과 변수를 전달하고, 'gs' 라는 객체로 만듦.
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
# cv: 폴드 수를 지정하는 매개변수. 기본값(매개변수 지정을 생략함)은 5이므로
# 지정한 매개변수 'min_impurity_decrease' 의 값마다 '5-폴드 교차 검증'을 수행함.
# 즉 '5 x 5 = 25' 개의 모델을 훈련함.
# n_job: 병렬 실행에 사용할 CPU 코어 수를 지정하는 매개변수. 기본값은 1이며, -1이면 모든 코어를 사용함.

In [77]:
# 'gs' 객체에 'fit()' 메서드를 호출하면 'min_impurity_decrease' 값을 바꿔가며 5번 실행함.
gs.fit(train_input, train_target)

# 교차 검증에서 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 하지만,
# 그리드 서치는 훈련이 끝나면 모델 25개 중에 검증 점수가 가장 높은 모델의 매개변수 조합으로
# 전체 훈련 세트에서 자동으로 다시 모델을 훈련함.
# 이 모델은 'best_estimator_' 속성에 저장됨.
# 최적의 매개변수 값은 'best_params_' 속성에 저장됨.
print(gs.best_estimator_) # 결정 트리 클래스에 매개변수의 최적값이 자동으로 입력됨.
print(gs.best_params_) # 'min_impurity_decrease' 매개변수의 최적값은 0.0001.

# 'dt' 객체에 'gs.best_estimator_' 속성을 그대로 전달하고 점수를 출력함.
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0001, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')
{'min_impurity_decrease': 0.0001}
0.9615162593804117


In [78]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수는 'cv_results_' 속성의 'mean_test_score' 키에 저장됨.
print(gs.cv_results_)
print(gs.cv_results_['mean_test_score'])
# 교차 검증의 평균 점수 순위는 'cv_results_' 속성의 'rank_test_score' 키에 저장됨.
print(gs.cv_results_['rank_test_score'])
# 첫 번째 값이 가장 큼.

# 넘파이 'argmax()' 함수를 사용하면 가장 큰 값의 인덱스를 추출함.
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(best_index)
print(gs.cv_results_['params'])
print(gs.cv_results_['params'][best_index])

{'mean_fit_time': array([0.01008415, 0.00961661, 0.0102632 , 0.00713315, 0.0063612 ]), 'std_fit_time': array([0.00218458, 0.00284024, 0.00529055, 0.00059508, 0.00099394]), 'mean_score_time': array([0.00095072, 0.00076957, 0.00077729, 0.00075984, 0.00078197]), 'std_score_time': array([2.02937693e-04, 2.64476603e-05, 5.98381048e-05, 3.45672408e-05,
       2.58948543e-04]), 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}], 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]), 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]), 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216,

#### 더 복잡한 매개변수 조합의 최적값 찾기

In [79]:
# 그리드 서치로 더 복잡한 매개변수 조합의 최적값을 찾음.
# 'min_impurity_decrease', 'max_depth', 'min_samples_split' 매개변수의 최적값을 찾기 위해
# 위와 동일하게 'params' 변수에 딕셔너리로 된 조합을 전달함.
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)
        }
# 넘파이 'arange()' 함수는 첫 번째 매개변수 값에서 두 번째 매개변수까지 세 번째 매개변수를 계속 더한 배열을 만듦.
print(np.arange(0.0001, 0.001, 0.0001)) # 두 번째 매개변수(0.001)는 끝지점이므로 포함되지 않음.
# 원소 9개가 만들어짐.

# 파이썬 'range()' 함수는 넘파이 'arange()' 함수와 비슷하며, 정수만 사용할 수 있음.
print(range(5, 20, 1))
print(np.array(range(5, 20, 1))) # 'np.array()' 함수를 사용하여 'range()' 함수의 내용물을 펼침.
# 원소 15개가 만들어짐.

print(range(2, 100, 10))
print(np.array(range(2, 100, 10)))
# 원소 10개가 만들어짐.

# 따라서 이 매개변수로 수행할 교차 검증 횟수는 '9 x 15 x 10 = 1,350회' 이며,
# 'cv' 매개변수의 기본값인 5로 '5-폴드 교차 검증' 을 수행하므로 모델은 '1,350 x 5 = 6,750개' 가 만들어짐.

[0.0001 0.0002 0.0003 0.0004 0.0005 0.0006 0.0007 0.0008 0.0009]
range(5, 20)
[ 5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
range(2, 100, 10)
[ 2 12 22 32 42 52 62 72 82 92]


In [80]:
# 그리드 서치를 실행함.
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)

gs.fit(train_input, train_target)
print(gs)

# 최상의 매개변수 조합을 확인함.
print(gs.best_params_)

# 최상의 교차 검증 점수를 확인함.
print(np.max(gs.cv_results_['mean_test_score']))

# 위에서 매개변수 간격을 '0.0001'과 '1'로 설정했으나 특별한 근거를 두고 설정한 값은 아님. 
# 더 좁거나 넓은 간격으로 시도할 수 있음. 
# 다만 매개변수 값이 수치일 때는 값의 범위나 간격을 미리 정하기 어려울 수 있고,
# 너무 많은 매개변수 조건이 있으면 그리드 서치 수행 시간이 오래 걸릴 수 있음.
# 이런 상황에서는 '랜덤 서치(Random search)'를 사용함. 아래에서 설명함.

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

### 랜덤 서치

- 매개변수 값의 목록을 전달하지 않고 매개변수를 샘플링 할 수 있는 '확률 분포 객체'를 전달함. 이 객체를 만들기 위해 '싸이파이(Scipy)'라는 파이썬 핵심 과학 라이브러리에 준비된 '확률 분포' 클래스를 활용함.

- 주어진 범위에서 고르게 값을 뽑는 클래스는 'randint' 와 'uniform' 이며, 이 두 클래스를 활용하여 '균등 분포에서 샘플링한다.' 라고 함. '난수 발생기'와 유사함. 샘플링 횟수는 시스템 자원이 허락하는 범위 내에서 최대한 크게 하는 것이 좋음.

 - randint: 정수값을 뽑는 클래스.

 - uniform: 실수값을 뽑는 클래스.

- 그리드 서치보다 교차 검증 수를 줄이면서 넓은 영역을 효과적으로 탐색 가능함.

- RandomizedSearchCV: 교차 검증으로 랜덤한 하이퍼파라미터를 탐색하는 클래스. 최상의 모델을 찾은 후 훈련 세트 전체를 사용하여 최종 모델을 훈련함. 교차 검증 함수(cross_validate())의 매개변수와 동일함.

 - scoring: 검증에 사용할 평가 지표를 지정하는 매개변수. 기본값은 분류 모델은 정확도인 'accuracy', 회귀 모델은 결정계수인 'r2'.

 - cv: 교차 검증 폴드 수나 스플리터 객체를 지정하는 매개변수. 기본값은 5. 자세한 설명은 아래에서 제시함.

 - n_jobs: 교차 검증을 수행할 때 사용할 CPU 코어 수를 지정하는 매개변수. 기본값은 1이며 -1로 지정하면 모든 시스템 코어를 사용함.

 - return_train_score: True로 지정하면 훈련 세트의 점수도 반환하는 매개변수. 기본값은 False.

In [81]:
# 주어진 범위에서 고르게 값을 뽑는 클래스를 부름.
from scipy.stats import randint, uniform

# 정수 10개를 샘플링 함.
rgen = randint(0, 10)
print(rgen.rvs(10)) # rvs: '랜덤 표본 생성(random variable sampling)' 횟수를 지정하는 함수.

[7 2 6 0 5 0 2 0 1 5]


In [82]:
# 정수 1000개를 샘플링 함.
print(rgen.rvs(1000))

# 각 정수의 개수를 셈.
np.unique(rgen.rvs(1000),  return_counts = True) # 'return_counts' 매개변수를 True로 지정하여 개수를 출력함.
# 0 ~ 9 까지의 정수가 각각 출력된 개수를 세서 배열에 맞게 출력함.
# 0은 102번 출력했고, 1은 111번 출력함.

[8 4 8 1 8 8 1 1 9 6 3 7 7 6 0 7 8 8 7 9 2 9 4 4 6 6 5 6 4 7 7 5 3 9 6 6 2
 7 9 6 9 7 3 7 4 4 3 1 8 9 1 5 3 3 0 6 3 2 4 7 0 0 1 4 3 3 3 5 6 2 4 4 1 9
 6 4 5 7 6 9 8 7 3 9 9 5 7 7 2 4 6 8 4 8 9 0 1 2 9 5 2 1 3 3 0 5 2 7 8 5 4
 0 3 2 0 5 7 4 1 5 0 0 2 1 8 1 4 6 4 6 3 6 7 4 4 4 2 8 0 9 7 8 2 4 2 2 2 1
 1 2 9 5 6 5 7 9 0 2 5 4 9 0 8 0 5 4 0 6 2 4 4 2 8 2 4 1 3 5 1 5 0 2 0 1 1
 2 3 0 4 0 0 1 3 2 4 3 3 2 8 2 6 0 8 4 8 0 0 1 0 6 7 7 1 8 5 7 2 4 8 8 4 8
 8 9 1 5 9 6 9 5 2 9 6 8 0 1 3 5 1 9 3 4 9 2 6 7 9 7 5 0 1 9 8 7 0 9 7 8 1
 3 6 5 5 7 7 7 6 3 2 3 2 3 3 3 9 1 1 7 7 1 7 3 2 3 8 3 8 0 0 7 6 8 7 7 0 2
 5 7 9 3 2 0 2 0 9 3 8 1 0 1 8 8 1 3 4 0 9 5 8 0 4 9 9 6 5 6 0 6 3 2 3 6 9
 9 7 8 1 6 8 0 6 9 0 5 4 6 6 7 2 4 5 4 1 9 2 8 7 1 5 6 0 7 4 6 2 0 1 3 0 9
 2 2 7 5 4 1 6 6 7 9 5 9 4 3 0 1 0 7 4 3 7 8 3 1 6 2 5 3 4 8 4 7 6 0 3 0 6
 6 6 4 4 9 1 0 7 4 2 8 6 5 9 2 0 8 4 1 7 3 5 9 0 1 2 2 0 8 6 0 2 4 4 2 1 9
 0 4 7 4 0 8 1 8 6 9 6 9 6 5 0 5 3 5 0 3 6 5 9 1 0 9 8 0 4 8 2 4 4 4 5 1 8
 7 6 8 7 9 2 0 7 2 2 3 3 

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([102, 111,  87, 107, 102, 104, 112,  97,  95,  83]))

In [83]:
# 실수 10개를 샘플링 함. 정수 샘플링 과정과 동일함.
ugen = uniform(0, 1)
print(ugen.rvs(10))

[0.27266143 0.77814826 0.36306631 0.99280009 0.73682509 0.83988546
 0.22131191 0.59237752 0.25478602 0.49716824]


#### 탐색할 매개변수의 딕셔너리 만들기

In [84]:
# 'min_samples_leaf' 매개변수를 탐색 대상에 추가함.
# min_samples_leaf: 리프 노드가 되기 위한 최소 샘플의 개수를 지정하는 매개변수이며,
# 어떤 노드가 분할하여 만들어질 자식 노드의 샘플 수가 이 값보다 작을 경우 분할하지 않음.
params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(20, 50),
          'min_samples_split' : randint(2, 25),
          'min_samples_leaf' : randint(1, 25)
        }

#### 샘플링 횟수 지정하기

In [85]:
# 샘플링 횟수는 'RandomizedSearchCV' 클래스의 'n_iter' 매개변수에 지정함.
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs = -1, random_state = 42)

#### 랜덤 서치로 훈련하기

In [87]:
# 랜덤 서치 클래스로 훈련 세트를 훈련함.
gs.fit(train_input, train_target)

# 최적의 매개변수 조합을 출력함.
print(gs.best_params_)

# 최고의 교차 검증 점수를 확인함.
print(np.max(gs.cv_results_['mean_test_score']))

# 최적의 모델을 확인하고 테스트 세트의 점수를 성능을 확인함.
dt = gs.best_estimator_
print(dt)
print(dt.score(test_input, test_target))

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=39, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.00034102546602601173,
                       min_impurity_split=None, min_samples_leaf=7,
                       min_samples_split=13, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=42, splitter='best')
0.86


## 마무리

마지막 RandomizedSearchCV 예제에서 DecisionTreeClassifier 클래스에 'splitter = random' 매개변수를 추가하고 다시 훈련함. 정리 중..